Import Module

In [1]:
import pandas as pd
import scanpy as sc
import scPAFA

Import AnnData contains cell-level metadata(normally store in AnnData.obs) 

In [2]:
#read_adata_and_select_tumor_cells
adata = sc.read_h5ad('/data/scPAFA_paper/CRC_file/20240218_cRc_geneover30cells_withobs_filtered.h5ad')

Import PAS matrix from step 1 and aligning the order with cell-level metadata

In [3]:
Ucell_result = pd.read_parquet('/data/scPAFA_paper/CRC_file/20231109_crc_2k.parquet')
Ucell_result = Ucell_result.loc[adata.obs.index]

Showing some information in anndata.obs

In [4]:
adata.obs[['pseudobulk_sample','ClusterFull','batch']][0:10]

,pseudobulk_sample,ClusterFull,batch
C103_T_1_1_0_c1_v2_id-AAACCTGCATGCTAGT,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACCTGGTAGCCTAT,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACCTGGTTGTCGCG,C103_T_v2,Tumor cE03 (StemTA-like prolif),v2
C103_T_1_1_0_c1_v2_id-AAACCTGTCATGTGGT,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACCTGTCCTTGGTC,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACCTGTCGTACGGC,C103_T_v2,Tumor cE03 (StemTA-like prolif),v2
C103_T_1_1_0_c1_v2_id-AAACCTGTCTTGCAAG,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACGGGAGACGCACA,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACGGGAGAGAACAG,C103_T_v2,Tumor cE01 (StemTA-like),v2
C103_T_1_1_0_c1_v2_id-AAACGGGAGTAGCCGA,C103_T_v2,Tumor cE03 (StemTA-like prolif),v2


## Using general MOFA framework (single-group framework)

Assigning PAS of pathways as features, cell types as non-overlapping sets of modalities (also called views) 

In [5]:
#step 2 
pseudobulk_table_single_group = scPAFA.pb.generate_scpafa_input(metadata=adata.obs,
                                                   PAS_dataframe=Ucell_result,
                                                    min_cell_number_per_sample=10,
                                                    min_sample_per_view=45,
                                                   sample_column='pseudobulk_sample',
                                                   view_column='ClusterFull',
                                                   top_percentage=0.25)

Select the top 408 pathways with the maximum variance
Done_Qc
Done_matrix
Done_longtable


In [6]:
pseudobulk_table_single_group[0:10]

,sample,feature,value,view
0,C103_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.285903,Tumor cE01 (StemTA-like)
1,C104_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.296543,Tumor cE01 (StemTA-like)
2,C105_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.356401,Tumor cE01 (StemTA-like)
3,C106_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.267785,Tumor cE01 (StemTA-like)
4,C107_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.439987,Tumor cE01 (StemTA-like)
5,C109_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.344231,Tumor cE01 (StemTA-like)
6,C110_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.568889,Tumor cE01 (StemTA-like)
7,C111_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.442938,Tumor cE01 (StemTA-like)
8,C112_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.272289,Tumor cE01 (StemTA-like)
9,C113_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.341200,Tumor cE01 (StemTA-like)


In [7]:
#step3
MOFA_model_single_group = scPAFA.pb.run_mofapy2(long_table=pseudobulk_table_single_group,
                                                scale_views=True,
                                                factor_number=10,
                                                random_seed=2023,
                                                spikeslab_weights=True,
                                                ard_weights=True,
                                                dropR2=0.01,
                                                convergence_mode='fast')


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\scPAFA        ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Scaling views to unit variance...


No "group" column found in the data frame, we will assume a common group for all samples...


Loaded group='single_group' view='Tumor cE01 (StemTA-like)' with N=65 samples and D=408 features...
Loaded group='single_group' view='Tumor cE02 (StemTA-likeImmature Goblet)' with N=55 samples and D=408 features...
Loaded group='single_group' view='Tu

Iteration 76: time=0.04, ELBO=-100574.09, deltaELBO=17.310 (0.00186149%), Factors=8
Iteration 77: time=0.04, ELBO=-100557.66, deltaELBO=16.434 (0.00176734%), Factors=8
Iteration 78: time=0.04, ELBO=-100542.05, deltaELBO=15.607 (0.00167836%), Factors=8
Iteration 79: time=0.04, ELBO=-100527.26, deltaELBO=14.796 (0.00159113%), Factors=8
Iteration 80: time=0.04, ELBO=-100513.15, deltaELBO=14.102 (0.00151656%), Factors=8
Iteration 81: time=0.04, ELBO=-100499.62, deltaELBO=13.533 (0.00145537%), Factors=8
Iteration 82: time=0.04, ELBO=-100486.56, deltaELBO=13.058 (0.00140427%), Factors=8
Iteration 83: time=0.04, ELBO=-100473.91, deltaELBO=12.654 (0.00136075%), Factors=8
Iteration 84: time=0.04, ELBO=-100461.60, deltaELBO=12.309 (0.00132368%), Factors=8
Iteration 85: time=0.04, ELBO=-100449.58, deltaELBO=12.016 (0.00129219%), Factors=8
Iteration 86: time=0.04, ELBO=-100437.82, deltaELBO=11.759 (0.00126458%), Factors=8
Iteration 87: time=0.04, ELBO=-100426.29, deltaELBO=11.532 (0.00124012%), Fa

In [8]:
#save model
MOFA_model_single_group.save('/data/scPAFA/example_file/20240218_CRC_mofa_model_single_group.hdf5')

Saving model in /data/scPAFA/example_file/20240218_CRC_mofa_model_single_group.hdf5...


## Using multi-group MOFA+ framework 

Assigning PAS of pathways as features, cell types as non-overlapping sets of modalities (also called views) and technical batch information as groups.

In [9]:
#step 2 
pseudobulk_table_multi_group = scPAFA.pb.generate_scpafa_input_multigroup(metadata=adata.obs,
                                                    PAS_dataframe=Ucell_result,
                                                    min_cell_number_per_sample=10,
                                                    min_percentage_sample_per_view=0.75,
                                                    min_sample_per_view=15,
                                                    sample_column='pseudobulk_sample',
                                                    view_column='ClusterFull',
                                                    group_column='batch',
                                                    top_percentage=0.25)

2 groups indentified
processing group v2
Select the top 408 pathways with the maximum variance
Done_Qc
Done_matrix
Done_longtable
processing group v3
Select the top 408 pathways with the maximum variance
Done_Qc
Done_matrix
Done_longtable


In [10]:
pseudobulk_table_multi_group[0:10]

,sample,feature,value,view,group
0,C103_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.285903,Tumor cE01 (StemTA-like),v2
1,C104_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.296543,Tumor cE01 (StemTA-like),v2
2,C105_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.356401,Tumor cE01 (StemTA-like),v2
3,C106_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.267785,Tumor cE01 (StemTA-like),v2
4,C107_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.439987,Tumor cE01 (StemTA-like),v2
5,C109_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.344231,Tumor cE01 (StemTA-like),v2
6,C110_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.568889,Tumor cE01 (StemTA-like),v2
7,C111_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.442938,Tumor cE01 (StemTA-like),v2
8,C112_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.272289,Tumor cE01 (StemTA-like),v2
9,C113_T_v2,bioplanet_1529_Endosomal/vacuolar pathway_View...,0.341200,Tumor cE01 (StemTA-like),v2


In [11]:
MOFA_model_multi_group = scPAFA.pb.run_mofapy2(long_table=pseudobulk_table_multi_group,
                                   scale_groups=True,
                                   scale_views=True,
                                   factor_number=10,
                                   random_seed=2023,
                                   spikeslab_weights=True,
                                   ard_factors = True,
                                   ard_weights=True,
                                   dropR2=0.01,
                                   convergence_mode='fast')


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\scPAFA        ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Scaling views to unit variance...

Scaling groups to unit variance...



Loaded group='v2' view='Tumor cE01 (StemTA-like)' with N=44 samples and D=408 features...
Loaded group='v2' view='Tumor cE02 (StemTA-likeImmature Goblet)' with N=36 samples and D=408 features...
Loaded group='v2' view='Tumor cE03 (StemTA-like prolif)' with N=44 samples and D=408 features...
Loaded group='v2

Iteration 71: time=0.05, ELBO=-114391.05, deltaELBO=36.982 (0.00409722%), Factors=8
Iteration 72: time=0.05, ELBO=-114354.54, deltaELBO=36.506 (0.00404444%), Factors=8
Iteration 73: time=0.05, ELBO=-114318.55, deltaELBO=35.992 (0.00398755%), Factors=8
Iteration 74: time=0.05, ELBO=-114283.10, deltaELBO=35.442 (0.00392661%), Factors=8
Iteration 75: time=0.05, ELBO=-114248.25, deltaELBO=34.853 (0.00386137%), Factors=8
Iteration 76: time=0.05, ELBO=-114214.03, deltaELBO=34.223 (0.00379147%), Factors=8
Iteration 77: time=0.05, ELBO=-114180.48, deltaELBO=33.544 (0.00371628%), Factors=8
Iteration 78: time=0.05, ELBO=-114147.68, deltaELBO=32.805 (0.00363439%), Factors=8
Iteration 79: time=0.05, ELBO=-114115.69, deltaELBO=31.985 (0.00354361%), Factors=8
Iteration 80: time=0.05, ELBO=-114084.63, deltaELBO=31.068 (0.00344199%), Factors=8
Iteration 81: time=0.05, ELBO=-114054.58, deltaELBO=30.049 (0.00332910%), Factors=8
Iteration 82: time=0.05, ELBO=-114025.63, deltaELBO=28.945 (0.00320679%), Fa

In [12]:
MOFA_model_multi_group.save('/data/scPAFA/example_file/20240218_CRC_mofa_model_multi_group.hdf5')

Saving model in /data/scPAFA/example_file/20240218_CRC_mofa_model_multi_group.hdf5...
